# Package and functions

In [1]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.octave import *
from qm.qua import *


#General os library 
import os
import time as time_package
import sys

sys.path.append(r"C:\Users\HQClabo\Documents\Code\QuantumMachine\Guillaume\NonLinearRes_calib_30_03_2023\QM_NonLinRes\Calibration_Script")
dll_path=r"C:\\Program Files\\Signal Hound\\Spike\\sa_api.dll"

import matplotlib.pyplot as plt
from qualang_tools.units import unit
from set_octave import get_L0_and_IF
#from configuration_first_jump_sticky import *
from configuration import *

from qm import SimulationConfig, LoopbackInterface
from qualang_tools.units import unit
from qualang_tools.loops import from_array
u = unit()
from qualang_tools.plot import interrupt_on_close
from qualang_tools.results import progress_counter, fetching_tool
from qualang_tools.addons.variables import assign_variables_to_element



#Path for HQCPC7 
sys.path.append(r'C:\Users\HQClabo\Documents\Code\QuantumMachine\Guillaume\NonLinearRes_calib_30_03_2023\Analysis_Function\NonLinearRes')
sys.path.append(r'C:\Users\HQClabo\Documents\Code\QuantumMachine\Guillaume\NonLinearRes_calib_30_03_2023\Drivers\Drivers') #path for the analysis function

#Path for HQC PC15 
# sys.path.append(r'C:\Users\hqclabo\Documents\Code\gbeaulieu\Analysis_Function\NonLinearRes')
# sys.path.append(r'C:\Users\hqclabo\Documents\Code\gbeaulieu\Drivers')

from lmfit.models import LinearModel
from Signal_hound_driver import Signalhound 
import Calibration_function as calib
import update_config as cfg


import AnFunc as an # analysis function
import imageio
import scipy.io
from scipy import signal
from scipy.optimize import curve_fit

In [2]:

def single_IQ_trace_without_plot_1(n_empty,n_full,IF_pump,IF_resonator,Offset_IF,amp_factor,Readout_Delay,Readout_Len,qm):
    
    """ gets the IQ values for a given pump frequency and amplitude 
    IF_Pump : If frequency of the pump 
    IF_resonator : IF frequency of the resonator (should be calculated before such that it gives half of the pump total frequency (LO_fluxline+IF_pump)/2-LO_readout
    Offset_IF : in case the down converted frequency is not perfectly centered around 0. This is typically kept to zero
    amp_facot amplitude factor of the pump
    n_runs : number of points 
    Readout_delay : waiting time before the first readout in clock cycle
    Readout_Len : time in ns of each readout 
    points_delay: waiting time between collected points 
    qm : quantum manager 

    returns :

    I: vector n_runs components of the frist quadrature
    Q : vector of n_runs compontents of the second quadrature
    time : vector of the time """

    waiting_time=16

    with program() as IQ_blobs:

        n = declare(int)
        i = declare(int)
        I = declare(fixed)
        Q = declare(fixed)
        assign_variables_to_element("resonator", I,Q) #This line forces the OPX to assign I and Q to the resonator element such that the loops can happen in parallel as intented
        I_st = declare_stream()
        Q_st = declare_stream()
        f = declare(int)
        
        it_numb_st=declare_stream()

        # Change the of the pump and resonator to demodulate better
        update_frequency("resonator",IF_resonator+Offset_IF) 
        update_frequency("fluxline",IF_pump)
          
        #Delay time before the first readout 
        wait(Readout_Delay,"resonator")
        
        with for_(i, 0, i < np.round(((n_full)*(Readout_Len+300)+Readout_Delay*4)/twoPhoton_len)+1, i + 1): # the 300 is added because there is a time delay associated to saving the data in the steam. The *4 is necessary because the waiting time is in clockcycle 
            play("pumping" * amp(amp_factor), 'fluxline')  # Play the pulse on the fluxline

        with for_(n, 0, n < n_full, n + 1):
            
            #Demodulate for the length 
            measure(
                "fake_readout",
                "resonator",
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )

            save(I, I_st)
            save(Q, Q_st)
            
            wait(waiting_time)
            
            
        align("resonator","fluxline")   
        wait(Readout_Delay,"resonator") 
            
        with for_(n, 0, n < n_empty, n + 1):

        #Demodulate for the length 
            measure(
                "fake_readout",
                "resonator",
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )

            save(I, I_st)
            save(Q, Q_st)    
            
            wait(waiting_time)
        
             

        with stream_processing():
            I_st.with_timestamps().save_all("I")
            Q_st.with_timestamps().save_all("Q")

    simulation = False
    if simulation:

        simulation_config = SimulationConfig(
            duration=200000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
        )
        job = qmm.simulate(config, IQ_blobs, simulation_config)
        job.get_simulated_samples().con1.plot(analog_ports={'1','2','3','4' },digital_ports={'1','3'})

    else: 



        job = qm.execute(IQ_blobs)
        results = fetching_tool(job, data_list=["I", "Q"], mode="live")
#        I, Q = results.fetch_all()
#        I = u.demod2volts(I["value"], Readout_Len) #diviser par la duree du pulse 
#        Q = u.demod2volts(Q["value"], Readout_Len)
        
        

 
        while results.is_processing():

            I, Q = results.fetch_all()

            time=I["timestamp"]
            I = u.demod2volts(I["value"], Readout_Len) #diviser par la duree du pulse 
            Q = u.demod2volts(Q["value"], Readout_Len)

    I, Q = results.fetch_all()
    time=I["timestamp"]
    I = u.demod2volts(I["value"], Readout_Len) #diviser par la duree du pulse 
    Q = u.demod2volts(Q["value"], Readout_Len)

      
    return I, Q




# OPX configuration

In [5]:
#Octave & OPX configuration 
Saving_Path=r"\\sb1files\HQC\Users\beaulieu\Data\Run01\131\\"
Cooldown_Date="2023_08_25"

calibration_path='C:\\Users\\HQClabo\\Documents\\Code\\QuantumMachine\\Guillaume\\NonLinearRes_calib_30_03_2023\\QM_NonLinRes\\Calibration_Script'


opx_ip = '128.178.175.167'
opx_port = 81
octave_ip = '128.178.175.167'
octave_port = 53


octave_config = QmOctaveConfig()
octave_config.set_calibration_db(calibration_path) #Path to the calibration database 

octave_config.add_device_info('octave1', octave_ip, octave_port) #Add a device refered to as octave 1
octave_config.set_opx_octave_mapping([('con1', 'octave1')])  # set default mapping between analog outputs of OPX and the octave


qmm = QuantumMachinesManager(host=opx_ip, port=opx_port, octave=octave_config)
qm = qmm.open_qm(config)


#Mixers for the fluxline 
octave_output_gain=-10

#Mixers for the fluxline 
qmm.octave_manager.set_clock("octave1", ClockType.External, ClockFrequency.MHZ_10) # External clock on the octave 

qm.octave.set_lo_source("fluxline", OctaveLOSource.Internal) # Use internal LO for the fluxline 
qm.octave.set_lo_frequency("fluxline", LO_fluxline)  # Set the frequency of the LO 
qm.octave.set_rf_output_gain("fluxline", octave_output_gain)  # can set gain from -10dB to 20dB
qm.octave.set_rf_output_mode("fluxline", RFOutputMode.on)  # The LO output is always on (could change to a trigger)  #RFOutputMode.trig_normal

#Mixers for the resonator 
qm.octave.set_qua_element_octave_rf_in_port("resonator", "octave1", 1) #input port 1 is set for the resonator 
qm.octave.set_downconversion("resonator", lo_source=RFInputLOSource.Internal)  # The LO for the demodulation is the interal LO  

qm.octave.set_lo_source("resonator", OctaveLOSource.Internal) # Use internal LO for the fluxline 
qm.octave.set_lo_frequency("resonator", LO_readout)  # Set the frequency of the LO 
qm.octave.set_rf_output_gain("resonator", octave_output_gain)  # can set gain from -10dB to 20dB
qm.octave.set_rf_output_mode("resonator", RFOutputMode.trig_normal)  # The LO output is always on (could change to a trigger)RFOutputMode.trig_normal


offset_1=0  #0.01783981341796875
offset_2=0  #0.01553899095214844
config,qm=cfg.update_offset(offset_1, offset_2,config,qmm)

2023-09-05 10:09:00,706 - qm - INFO - Performing health check
2023-09-05 10:09:00,727 - qm - WARNING -   HC Warning: Potential hardware issue with QSync in controller con1, multiple controllers sync cannot be guaranteed, please inform QM
2023-09-05 10:09:00,728 - qm - INFO - Health check passed


# Display the parameters 

In [80]:
print("LO of resonator : {} GHz".format(config['elements']['resonator']["mixInputs"]["lo_frequency"]/1e9))
print("IF of resonator : {} GHz".format(config['elements']['resonator']['intermediate_frequency']/1e9))
print("Time of flight of resonator : {} ns".format(config['elements']['resonator']['time_of_flight']))
print("Readout time resonator : {} ns".format(config["pulses"]["zero_pulse"]["length"]))
print("LO of fluxline: {} GHz".format(config['elements']['fluxline']["mixInputs"]["lo_frequency"]/1e9))
print("IF of fluxline : {} GHz".format(config['elements']['fluxline']['intermediate_frequency']/1e9))
print("Fluxline pulse time : {} ns".format(config["pulses"]["twoPhoton"]["length"]))
print("Fluxline amplitude : {} V".format(config["waveforms"]["twoPhoton_wf"]['sample']))

LO of resonator : 4.2 GHz
IF of resonator : 0.1 GHz
Time of flight of resonator : 24 ns
Readout time resonator : 50000 ns
LO of fluxline: 8.6 GHz
IF of fluxline : 0.0995 GHz
Fluxline pulse time : 10000 ns
Fluxline amplitude : 0.185 V


# ADC 

In [6]:
#Sets the pulse length and the pulse amplitude at the resonator 
Readout_Len=50_000
config,qm=cfg.update_readout_lenght(Readout_Len,config,qmm)

# twoPhoton_len=100_000
# config,qm=update_two_photon_lenght(twoPhoton_len,config,qmm)


amp_factor=0 #Amplitude factor of the pump 

n_avg = 1000  # Number of averaging loops

IF_pump=99471960 # if to be at twice the resoance frequency 
IF_resonator=(LO_fluxline+IF_pump)/2-LO_readout

Readout_Delay=10_000//4 # Delay before the first readout in ns (convert from clockcycle)
points_delay=16//4

offset_1=0.017572669677734375
offset_2=0.01528224609375
config,qm=cfg.update_offset(offset_1, offset_2,config,qmm)

In [21]:
%matplotlib qt

with program() as raw_trace_prog:
    
    n = declare(int)
    adc_st = declare_stream(adc_trace=True)
    i = declare(int)
    
    update_frequency("resonator",IF_resonator) 
    update_frequency("fluxline",IF_pump)
    wait(Readout_Delay,"resonator")
    
    

    with for_(i, 0, i < np.round((n_avg*(Readout_Len+300+points_delay*4)+Readout_Delay*4)/twoPhoton_len)+1, i + 1): # the 300 is added because there is a time delay associated to saving the data in the steam. The *4 is necessary because the waiting time is in clockcycle 
        play("pumping" * amp(amp_factor), 'fluxline')  # Play the pulse on the fluxline
        
    with for_(n, 0, n < n_avg, n + 1):
        
        #play("pumping"*amp(amp_factor), 'fluxline')
        #Delay time before the first readout 
        measure("fake_readout", "resonator", adc_st)
        
        wait(points_delay,"resonator")

    with stream_processing():
        
        # Will save average:
        adc_st.input1().average().save("adc1")
        adc_st.input2().average().save("adc2")
        
        # Will save only last run:
        adc_st.input1().save("adc1_single_run")
        adc_st.input2().save("adc2_single_run")

        
simulation = True
if simulation:
    
    simulation_config = SimulationConfig(
        duration=80000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
    )
    job = qmm.simulate(config, raw_trace_prog, simulation_config)
    job.get_simulated_samples().con1.plot(analog_ports={'1','2','3','4' },digital_ports={'1','3'})
    
    
    
else: 
    #Execute the program     
    qm = qmm.open_qm(config)
    job = qm.execute(raw_trace_prog)

    results = fetching_tool(job, data_list=["adc1", "adc2", "adc1_single_run","adc2_single_run"], mode="wait_for_all")

    adc1, adc2, adc1_single_run, adc2_single_run = results.fetch_all()

    adc1 = u.raw2volts(adc1)
    adc2 = u.raw2volts(adc2)
    adc1_single_run = u.raw2volts(adc1_single_run)
    adc2_single_run = u.raw2volts(adc2_single_run)

    #Plotting 
    plt.figure()
    plt.subplot(121)
    plt.title("Single run")
    plt.plot(adc1_single_run, label="Input 1")
    plt.plot(adc2_single_run, label="Input 2")
    plt.xlabel("Time [ns]")
    plt.ylabel("Signal amplitude [V]")
    plt.legend()

    plt.subplot(122)
    plt.title("Averaged run")
    plt.plot(adc1, label="Input 1")
    plt.plot(adc2, label="Input 2")
    plt.xlabel("Time [ns]")
    plt.legend()
    plt.tight_layout()

    print("The mean offset of port 1 is: {}".format(np.mean(adc1)))
    print("The mean offset of port 2 is: {}".format(np.mean(adc2)))

2023-09-03 09:43:06,548 - qm - INFO - Flags: 
2023-09-03 09:43:06,548 - qm - INFO - Simulating program


In [7]:
def vacuum_calculation(I,Q,G):
    " I, Q measured at the mixer and G the gain of the amplifier"
    
    I=np.sqrt(1/G)*I
    Q=np.sqrt(1/G)*Q
    
    
    S=(I+1j*Q)
    
    
    hd=np.mean(S) #<hdag>
    hhd=np.mean(S.conj()*S) # <h hdag>  
    hdhd=np.mean(S**2) 
    
    return hd, hhd,hdhd 

def field_calculation(I,Q,G,hd,hhd,hdhd):
    " I, Q measured at the mixer and G the gain of the amplifier"
    
    I=np.sqrt(1/G)*I
    Q=np.sqrt(1/G)*Q
    
    S=(I+1j*Q)
    
    
    a=np.mean(S)-hd
    aa=np.mean(S**2)-hdhd-2*a*hd
    ada=np.mean(S.conj()*S)-hhd-a*hd.conj()-a.conj()*hd
    
    noisy=np.mean(S.conj()*S)
   
    
    return a, ada, aa, noisy

def noise_average(a, n=1) :
    """
    This function does a static average of a vector , in order to reduce noise in too noisy signals.
    a: vector to average 
    n : number of points using in the average. It should be a divided of the vector length 
    
    returns : avgResult : averaged vector 
    """    
    if len(a)%n != 0:
        a = np.append(a, a[-1]*np.ones(n-len(a)%n))
        
    avgResult = np.average(a.reshape(-1, n), axis=1)
    return avgResult

# Multiple frequencies for second-order transition

In [8]:
#Select the frequencies 
freqs_dict={}

amp_factor_array=np.array([1.15]) #Prefactor of the amplitude 
amp_factor=amp_factor_array[0]

amp_factor_2=0

fmin=99.4e6
fmax=101e6
df=0.01* u.MHz
freqs=np.arange(fmin, fmax+df/2, df)


#freqs=np.array([100e6])

freqs_dict[amp_factor]=freqs
print(freqs)

[9.9400e+07 9.9410e+07 9.9420e+07 9.9430e+07 9.9440e+07 9.9450e+07
 9.9460e+07 9.9470e+07 9.9480e+07 9.9490e+07 9.9500e+07 9.9510e+07
 9.9520e+07 9.9530e+07 9.9540e+07 9.9550e+07 9.9560e+07 9.9570e+07
 9.9580e+07 9.9590e+07 9.9600e+07 9.9610e+07 9.9620e+07 9.9630e+07
 9.9640e+07 9.9650e+07 9.9660e+07 9.9670e+07 9.9680e+07 9.9690e+07
 9.9700e+07 9.9710e+07 9.9720e+07 9.9730e+07 9.9740e+07 9.9750e+07
 9.9760e+07 9.9770e+07 9.9780e+07 9.9790e+07 9.9800e+07 9.9810e+07
 9.9820e+07 9.9830e+07 9.9840e+07 9.9850e+07 9.9860e+07 9.9870e+07
 9.9880e+07 9.9890e+07 9.9900e+07 9.9910e+07 9.9920e+07 9.9930e+07
 9.9940e+07 9.9950e+07 9.9960e+07 9.9970e+07 9.9980e+07 9.9990e+07
 1.0000e+08 1.0001e+08 1.0002e+08 1.0003e+08 1.0004e+08 1.0005e+08
 1.0006e+08 1.0007e+08 1.0008e+08 1.0009e+08 1.0010e+08 1.0011e+08
 1.0012e+08 1.0013e+08 1.0014e+08 1.0015e+08 1.0016e+08 1.0017e+08
 1.0018e+08 1.0019e+08 1.0020e+08 1.0021e+08 1.0022e+08 1.0023e+08
 1.0024e+08 1.0025e+08 1.0026e+08 1.0027e+08 1.0028e+08 1.0029

In [9]:
%matplotlib qt 

#Update the readout pulse 
Readout_Len=2_000
config,qm=cfg.update_readout_lenght(Readout_Len,config,qmm)

twoPhoton_len=10_000
config,qm=cfg.update_two_photon_lenght(twoPhoton_len,config,qmm)

n_empty=10_000_000 # Number of runs without the pump  
n_full=10_000_000
n_runs=n_empty+n_full


Readout_Delay =100_000 


Offset_IF=0  #
number_rep = 1 #Number of times to reapeat the same measurement 


save_Data=True
#Saving the data
cooldown_date=Cooldown_Date #top folder 
device="second_order_IQ" #second folder 
name="1p15_2000_10_000_000_3" # third folder 
meastype="1p15" 

print(amp_factor)

print("The different frequency {}".format(freqs))
Expected_time=((Readout_Len)*n_runs*len(freqs))*1e-9/60
print("The expected time : {} min".format(Expected_time))
calib_dict={}

1.15
The different frequency [9.9400e+07 9.9410e+07 9.9420e+07 9.9430e+07 9.9440e+07 9.9450e+07
 9.9460e+07 9.9470e+07 9.9480e+07 9.9490e+07 9.9500e+07 9.9510e+07
 9.9520e+07 9.9530e+07 9.9540e+07 9.9550e+07 9.9560e+07 9.9570e+07
 9.9580e+07 9.9590e+07 9.9600e+07 9.9610e+07 9.9620e+07 9.9630e+07
 9.9640e+07 9.9650e+07 9.9660e+07 9.9670e+07 9.9680e+07 9.9690e+07
 9.9700e+07 9.9710e+07 9.9720e+07 9.9730e+07 9.9740e+07 9.9750e+07
 9.9760e+07 9.9770e+07 9.9780e+07 9.9790e+07 9.9800e+07 9.9810e+07
 9.9820e+07 9.9830e+07 9.9840e+07 9.9850e+07 9.9860e+07 9.9870e+07
 9.9880e+07 9.9890e+07 9.9900e+07 9.9910e+07 9.9920e+07 9.9930e+07
 9.9940e+07 9.9950e+07 9.9960e+07 9.9970e+07 9.9980e+07 9.9990e+07
 1.0000e+08 1.0001e+08 1.0002e+08 1.0003e+08 1.0004e+08 1.0005e+08
 1.0006e+08 1.0007e+08 1.0008e+08 1.0009e+08 1.0010e+08 1.0011e+08
 1.0012e+08 1.0013e+08 1.0014e+08 1.0015e+08 1.0016e+08 1.0017e+08
 1.0018e+08 1.0019e+08 1.0020e+08 1.0021e+08 1.0022e+08 1.0023e+08
 1.0024e+08 1.0025e+08 1.0026e+08

In [11]:
calib_dict={}

calib_amp_array_res=np.array([0.1, 0.5,1])
calib_amp_array_pump=np.array([0.85,0.95,1.05,1.15,1.25])

calib_amp_pump, calib_amp_res=calib.full_power_calibration(qm,dll_path,calib_amp_array_res, calib_amp_array_pump, IF_readout, LO_readout, IF_fluxline, LO_fluxline)

calib_dict["calib_amp_array_res"]=calib_amp_array_res
calib_dict["calib_amp_array_pump"]= calib_amp_array_pump
calib_dict["calib_amp_pump"]=calib_amp_pump
calib_dict["calib_amp_res"]=calib_amp_res

Unable to open device SignalHound
During saConfigCenterSpan, the error -3 was raised
During saConfigLevel, the error -3 was raised
During saConfigGainAtten, the error -3 was raised
During saConfigIQ, the error -3 was raised
During saConfigProcUnits, the error -3 was raised
During saQuerySweepInfo, the error -3 was raised
During saInitiate, the error -3 was raised
2023-08-31 22:29:36,435 - qm - INFO - Flags: 
2023-08-31 22:29:36,435 - qm - INFO - Sending program to QOP
2023-08-31 22:29:36,810 - qm - INFO - Executing program
0


KeyboardInterrupt: 

In [10]:
%%time

Data={} #dictionnary to hold all the data 
Data[amp_factor]={}
freqs=freqs_dict[amp_factor]



freqs=freqs_dict[amp_factor]
calib_dict["res_freq"]={}
    

for idx,IF_pump in enumerate(freqs) :
    
    #Frequency calibration
    calib_pulse_len=50_000
    config,qm=cfg.update_readout_lenght(calib_pulse_len,config,qmm)

    calib_n_avg = 1000

    calib_cooldown_time = 50_000 // 4

    calib_f_min =149.75e6-0.25e6
    calib_f_max =149.75e6+0.25e6
    calib_df = 0.0025e6   #0.001e6 #0.001e6
    calib_amp_factor=0.01 #amplitude factor 

    calib_freqs,calib_I,calib_Q,calib_resonance_frequency, calib_error_frequency=calib.SingleResonatorSpec(qm,calib_n_avg,calib_pulse_len, calib_cooldown_time, calib_f_min, calib_f_max, calib_df, calib_amp_factor,LO_readout,plot=True)
    plt.close()
    
    calib_dict["res_freq"][IF_pump]={}
    calib_dict["res_freq"][IF_pump]["calib_freqs"]=calib_freqs
    calib_dict["res_freq"][IF_pump]["calib_I"]= calib_I
    calib_dict["res_freq"][IF_pump]["calib_Q"]=calib_Q
    calib_dict["res_freq"][IF_pump]["calib_resonance_frequency"]=calib_resonance_frequency
    calib_dict["res_freq"][IF_pump]["calib_error_frequency"]=calib_error_frequency

    #re update the readout length 
    config,qm=cfg.update_readout_lenght(Readout_Len,config,qmm)
    
    print("########################### Frequency "+str(IF_pump+LO_fluxline)+"########################### "+str(idx/len(freqs)))

    IF_resonator=(LO_fluxline+IF_pump)/2-LO_readout #demodulate at half of the pump IF by default 
    
    
    for rep in range(number_rep):
        
        print("Average {}".format(rep/number_rep))
    
    
        I,Q=single_IQ_trace_without_plot_1(n_empty,n_full,IF_pump,IF_resonator,Offset_IF,amp_factor,Readout_Delay,Readout_Len,qm)

        plt.close()

        Data={} #dictionnary to hold all the data 
        Data[amp_factor]={}
        Data[amp_factor][IF_pump]={}
        Data[amp_factor][IF_pump]["I"]=I
        Data[amp_factor][IF_pump]["Q"]=Q


        if save_Data:
            datadict={}
            meastype_n=meastype+"_"+"avg"+"_"+str(rep)+"_" +str(IF_pump)

            datadict["freqs_dict"]=freqs_dict
            datadict["Readout_Len"]=Readout_Len
            datadict["Readout_Delay"]=Readout_Delay*4
            datadict["octave_output_gain"]=octave_output_gain
            datadict["amp_factor_array"]=amp_factor_array
            datadict["Data"]=Data
            datadict["config"]=config
            
            datadict["number_rep"]=number_rep
            datadict["n_empty"]=n_empty
            datadict["n_full"]=n_full

            #Configuration file copy 
            datadict["LO_fluxline"]=LO_fluxline
            datadict["IF_readout"]=IF_resonator
            datadict["LO_readout"]=LO_readout

            datadict["res_freq"]=calib_resonance_frequency


            #datadict["calib"]=calib_dict
            # datadict["calib_amp_pump"]=calib_amp_pump
            # datadict["calib_amp_res"]=calib_amp_res
            datadict["calib_freqs"]=calib_freqs
            datadict["calib_I"]= calib_I
            datadict["calib_Q"]=calib_Q


            an.save_with_numpy(datadict,  meastype_n, name, device, cooldown_date, filepath=Saving_Path)
            
            if (idx==41) or (idx==81) or (idx==120):
                print("CLEAR")
                qmm.clear_all_job_results()
                time_package.sleep(60)
            
            #
            time_package.sleep(10)
           
          


2023-09-05 10:09:21,042 - qm - INFO - Flags: 
2023-09-05 10:09:21,042 - qm - INFO - Sending program to QOP
2023-09-05 10:09:25,214 - qm - INFO - Executing program
Progress: [##################################################] 100.0% (n=1000/1000) --> elapsed time: 20.04s
The measured resonance frequency is 4.349719733267181 GHz with an error of 8.816445624407648e-07 GHz
########################### Frequency 8699400000.0########################### 0.0
Average 0.0
2023-09-05 10:09:45,894 - qm - INFO - Flags: 
2023-09-05 10:09:45,894 - qm - INFO - Sending program to QOP
2023-09-05 10:09:46,804 - qm - INFO - Executing program
saving in new directory
2023-09-05 10:11:12,233 - qm - INFO - Flags: 
2023-09-05 10:11:12,234 - qm - INFO - Sending program to QOP
2023-09-05 10:11:13,866 - qm - INFO - Executing program
Progress: [##################################################] 100.0% (n=1000/1000) --> elapsed time: 20.03s
The measured resonance frequency is 4.349723291548321 GHz with an error of

In [93]:
qmm.clear_all_job_results()

In [25]:
161*10/60

26.833333333333332

## Processing

In [12]:
f=100000000.0
av=0
amp_factor_str="1p15"
amp_factor=1.15
folder="1p15_2000_10_000_000_3"


filename=amp_factor_str+"_"+"avg_"+str(av)+"_"+str(f)
filepath= r"X:\Users\beaulieu\Data\Run01\131\2023_08_25\second_order_IQ\{}\{}".format(folder,filename+".npz")
npzfile=np.load(filepath,allow_pickle=True)
npzfile["multidata"][()]
Data=npzfile["multidata"][()]["Data"]
res_freq=npzfile["multidata"][()]["res_freq"]
freqs=npzfile["multidata"][()]["freqs_dict"][amp_factor]
number_rep=npzfile["multidata"][()]['number_rep']
n_full=npzfile["multidata"][()]['n_full']
n_empty=npzfile["multidata"][()]['n_empty']


In [13]:
G=1/50

sq_arr=[]
detuning_arr=[]
ada_arr=[]
hhd_arr=[]
hd_arr=[]
hdhd_arr=[]
aa_arr=[]
a_arr=[]
mean_x_arr=[]
mean_x_square_arr=[]
noisy_arr=[]


plot=False



for idx in range(len(freqs)):
    

    
    f=freqs[idx]
    
    print(f)
    
    detuning_arr.append(-(f+8.6e9-res_freq*2)/1e6/amp_factor)
    
    for av in range(number_rep):
    

        filename=amp_factor_str+"_"+"avg_"+str(av)+"_"+str(f)
        

        filepath= r"X:\Users\beaulieu\Data\Run01\131\2023_08_25\second_order_IQ\{}\{}".format(folder,filename+".npz")
        FileName=r"X:\Users\beaulieu\Data_Analysis\run01\131\2023_07_10\Plotting\squeezing\{}".format(folder)


        npzfile=np.load(filepath,allow_pickle=True)
        npzfile["multidata"][()]
        Data=npzfile["multidata"][()]["Data"]
        res_freq=npzfile["multidata"][()]["res_freq"]
        freqs=npzfile["multidata"][()]["freqs_dict"][amp_factor]


        I=Data[amp_factor][f]["I"]
        Q=Data[amp_factor][f]["Q"]


        I_vac=I[n_full:]*1e3
        Q_vac=Q[n_full:]*1e3

        I_sig=I[0:n_full]*1e3
        Q_sig=Q[0:n_full]*1e3


        if plot==True:

            fig, axs = plt.subplots( nrows=1, ncols=1, figsize=(5,5),dpi=100)

            axs.hist2d(I_sig,Q_sig,bins=100)
            axs.set_title(str(idx))

            plt.savefig(FileName+f'/img_{idx}.png', 
                                transparent = False,  
                                facecolor = 'white'
                               )    
            plt.close()



        #vacuum : 
        hd, hhd,hdhd=vacuum_calculation(I_vac,Q_vac,G)
        
        hhd_arr.append(hhd)
        hd_arr.append(hd)
        hdhd_arr.append(hdhd)
        
        #remove the vacuum : 
        a, ada, aa, noisy=field_calculation(I_sig,Q_sig,G,hd,hhd,hdhd)

        ada_arr.append(ada)


        #angle  
        phi=np.linspace(-np.pi,np.pi,100)

        
        #Calculate the mean of x the quadrature 
        mean_x=a*np.exp(-1j*phi)+a.conj()*np.exp(1j*phi)
        mean_x_arr.append(np.min(mean_x))
    
        mean_x_square=(aa*np.exp(-1j*phi)+aa.conj()*np.exp(1j*phi)+2*ada+1)/2
        mean_x_square_arr.append(np.min(mean_x_square))
        

        #squeezing calucaltion 
        sq=aa*np.exp(-1j*phi)+aa.conj()*np.exp(1j*phi)+2*ada+1-a**2*np.exp(-1j*phi)-a.conj()**2*np.exp(1j*phi)-2*a*a.conj()


        aa_arr.append(aa)
        a_arr.append(a)
        noisy_arr.append(noisy)

        #Select the minimum squeezing 
        sq_arr.append(np.min(sq))
        
        
mean_x_square_ar=np.array(mean_x_square_arr).reshape((int(len(mean_x_square_arr)/number_rep),number_rep))
mean_x_ar=np.array(mean_x_arr).reshape((int(len(mean_x_arr)/number_rep),number_rep)) 
ada_ar=np.array(ada_arr).reshape((int(len(ada_arr)/number_rep),number_rep)) 
a_ar=np.array(a_arr).reshape((int(len(a_arr)/number_rep),number_rep)) 
aa_ar=np.array(aa_arr).reshape((int(len(aa_arr)/number_rep),number_rep)) 


noisy_ar=np.array(noisy_arr).reshape((int(len(noisy_arr)/number_rep),number_rep)) 


sq_ar=np.array(sq_arr).reshape((int(len(sq_arr)/number_rep),number_rep)) 

hd_ar=np.array(hd_arr).reshape((int(len(hd_arr)/number_rep),number_rep)) 
hhd_ar=np.array(hhd_arr).reshape((int(len(hhd_arr)/number_rep),number_rep)) 
hdhd_ar=np.array(hdhd_arr).reshape((int(len(hdhd_arr)/number_rep),number_rep)) 




99400000.0
99410000.0
99420000.0
99430000.0
99440000.0
99450000.0
99460000.0
99470000.0
99480000.0
99490000.0
99500000.0
99510000.0
99520000.0
99530000.0
99540000.0
99550000.0
99560000.0
99570000.0
99580000.0
99590000.0
99600000.0
99610000.0
99620000.0
99630000.0
99640000.0
99650000.0
99660000.0
99670000.0
99680000.0
99690000.0
99700000.0
99710000.0
99720000.0
99730000.0
99740000.0
99750000.0
99760000.0
99770000.0
99780000.0
99790000.0
99800000.0
99810000.0
99820000.0
99830000.0
99840000.0
99850000.0
99860000.0
99870000.0
99880000.0
99890000.0
99900000.0
99910000.0
99920000.0
99930000.0
99940000.0
99950000.0
99960000.0
99970000.0
99980000.0
99990000.0
100000000.0
100010000.0
100020000.0
100030000.0
100040000.0
100050000.0
100060000.0
100070000.0
100080000.0
100090000.0
100100000.0
100110000.0
100120000.0
100130000.0
100140000.0
100150000.0
100160000.0
100170000.0
100180000.0
100190000.0
100200000.0
100210000.0
100220000.0
100230000.0
100240000.0
100250000.0
100260000.0
100270000.0
1002

In [44]:
print("The means is {}".format(np.mean(mean_x_square_ar)))
print("The std is {}".format(np.std(mean_x_square_ar)))

The means is (0.47533854186980157+0j)
The std is 0.019868035088235585


In [20]:
I.keys()

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [57]:
np.std(mean_x_square_ar)

0.04610409976139645

In [37]:
mean_x_square_ar

array([[0.48781211+0.j, 0.49845148+0.j, 0.50720695+0.j, 0.48844666+0.j,
        0.4703738 +0.j]])

In [29]:
mean_x_square_ar

array([[0.50347182+0.j, 0.493424  +0.j, 0.48669842+0.j, 0.48839514+0.j,
        0.50888835+0.j]])

In [54]:
mean_x_square_ar.shape

(161, 1)

In [102]:
plt.plot(detuning_arr[0:46],mean_x_square_arr,".")

In [16]:
fig, axs = plt.subplots( nrows=4, ncols=1, figsize=(8,16),dpi=100)


idx=0

detuning_arr_1=freqs
#detuning_arr_1=detuning_arr


#xlim=[-0.4,0.2]

#ylim=[-4,4]

axs[0].plot( detuning_arr_1, mean_x_square_ar,".",ms=10)
axs[0].set_xlabel("freq")
axs[0].set_ylabel("<x^{2}>")
#axs[0].set_xlim(xlim)

#axs[0].set_ylim(ylim)

axs[1].plot( detuning_arr_1, ada_ar[:,idx],".",label="ada")
#axs[1].plot( detuning_arr_1, hhd_ar[:,idx],".",label="hhd")
axs[1].legend()

axs[1].set_xlabel("freq")
axs[1].set_ylabel("n")


axs[2].plot( detuning_arr_1,detuning_arr ,".")

axs[2].set_xlabel("freqs")
axs[2].set_ylabel("Detuning")


axs[3].plot( detuning_arr_1, a_ar[:,idx],".")

axs[3].set_xlabel("freq")
axs[3].set_ylabel("a")

Text(0, 0.5, 'a')

In [197]:
fig, axs = plt.subplots( nrows=4, ncols=1, figsize=(8,16),dpi=100)


idx=0

detuning_arr_1=freqs

#xlim=[-0.4,0.2]

#ylim=[-4,4]

axs[0].plot( detuning_arr, mean_x_square_ar,".-",ms=10)
axs[0].set_xlabel("freq")
axs[0].set_ylabel("<x^{2}>")
#axs[0].set_xlim(xlim)

#axs[0].set_ylim(ylim)

axs[1].plot( detuning_arr, ada_ar[:,idx],".",label="ada")
#axs[1].plot( detuning_arr_1, hhd_ar[:,idx],".",label="hhd")
axs[1].legend()

axs[1].set_xlabel("freq")
axs[1].set_ylabel("n")


axs[2].plot( detuning_arr,detuning_arr ,".")

axs[2].set_xlabel("freqs")
axs[2].set_ylabel("Detuning")


axs[3].plot( detuning_arr, a_ar[:,idx],".")

axs[3].set_xlabel("freq")
axs[3].set_ylabel("a")

Text(0, 0.5, 'a')

In [23]:
  
datadict={}


cooldown_date="2023_07_10" #top folder 
Saving_Path=r"X:\Users\beaulieu\Data_Analysis\run01\131\2023_07_10\Data\\"

device="squeezing" #second folder 
name="1p15_2000_10_000_000_3" # third folder 
meastype="1p15" 

datadict["detuning"]=detuning_arr
datadict["freqs"]=freqs
datadict["n_empty"]=n_empty
datadict["n_full"]=n_full
datadict["n_rep"]=1

datadict["mean_x_square_ar"]=mean_x_square_ar

datadict["mean_x_ar"]=mean_x_ar

datadict["ada_ar"]=ada_ar

datadict["a_ar"]=a_ar

datadict["aa_ar"]=aa_ar

datadict["hd_ar"]=hd_ar
datadict["hhd_ar"]=hhd_ar
datadict["hdhd_ar"]=hdhd_ar

datadict["noisy_ar"]=hdhd_ar
        

an.save_with_numpy(datadict,  meastype_n, name, device, cooldown_date, filepath=Saving_Path)

saving in new directory


In [19]:
detuning_arr.shape()

AttributeError: 'list' object has no attribute 'shape'